In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-1/CGUHEmYUVq.pdf
/kaggle/input/test-1/aMF857DsyF.pdf


In [6]:
!pip install PyPDF2 pdfplumber pandas
!pip install marker-pdf
!pip install langchain_community
!pip install langchain_groq
!pip install cassio
!pip install groq
!pip install langchain_experimental
!pip install pandasai

  Using cached grpcio-1.64.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
Using cached grpcio-1.64.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.60.0
    Uninstalling grpcio-1.60.0:
      Successfully uninstalled grpcio-1.60.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1

In [7]:
import pdfplumber
import pandas as pd
from transformers import AutoTokenizer 
from langchain.vectorstores import FAISS 
from langchain.chains import RetrievalQA 
from langchain.chains import ConversationalRetrievalChain
import transformers 
import torch 
import textwrap 
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
import cassio
from dotenv import load_dotenv
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
load_dotenv()
import os
from groq import Groq
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_experimental.agents import create_pandas_dataframe_agent
from pandasai import SmartDataframe

In [8]:
# Function to extract table from PDF
def extract_table_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract table data
            table = page.extract_table()
            if table:
                tables.append(table)
    return tables

# Function to clean and convert extracted data to DataFrame
def tables_to_dataframe(tables):
    # Assuming the first row of the first table contains the header
    headers = tables[0][0]
    data = []

    for table in tables:
        for row in table[1:]:  # Skip the header row
            data.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)
    return df

# Function to save DataFrame to CSV
def save_to_csv(df, output_csv_path):
    df.to_csv(output_csv_path, index=False)

# Main function
def main(pdf_path, output_csv_path):
    tables = extract_table_from_pdf(pdf_path)
    df = tables_to_dataframe(tables)
    save_to_csv(df, output_csv_path)

# Path to input PDF and output CSV
pdf_path = '/kaggle/input/test-1/CGUHEmYUVq.pdf'
output_csv_path = 'output_1.csv'

# Run the main function
main(pdf_path, output_csv_path)


In [9]:
# Path to input PDF and output CSV
pdf_path = '/kaggle/input/test-1/aMF857DsyF.pdf'
output_csv_path = 'output_2.csv'

# Run the main function
main(pdf_path, output_csv_path)


In [10]:
llm_groq = ChatGroq(
            groq_api_key='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', model_name="mixtral-8x7b-32768",
                         temperature=0.2)

In [11]:
chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key="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" # Optional if not set as an environment variable
)

In [12]:
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [13]:
df=pd.read_csv('/kaggle/working/output_2.csv')

In [14]:
parser = PandasDataFrameOutputParser(dataframe=df)

In [15]:
df.head()

,Sr No.,Date of\nEncashment,Name of the Political Party,Account no. of\nPolitical Party,Prefix,Bond\nNumber,Denominations,Pay Branch\nCode,Pay Teller
0,1,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,775,"1,00,00,000",800,2770121
1,2,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3975,"1,00,00,000",800,2770121
2,3,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3967,"1,00,00,000",800,2770121
3,4,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,10418,"10,00,000",800,2770121
4,5,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,126,"10,00,000",800,2770121


In [41]:
m=pd.read_csv('/kaggle/working/output_1.csv')
k=pd.read_csv('/kaggle/working/output_2.csv')
m=m.rename(columns={'Date of\nPurchase':'Date of Purchase','Bond\nNumber':'Bond Number'})
k=k.rename(columns={'Date of\nEncashment':'Date of Encashment','Bond\nNumber':'Bond Number','Pay Branch Code':'Pay Branch Code'})



In [42]:
frame=k.merge(m,on=['Bond Number','Prefix'],how='left',suffixes=('', '_m'))

frame.fillna(0,inplace=True)

In [43]:
frame['Date of Encashment']=pd.to_datetime(frame['Date of Encashment'])
frame['Journal Date']=pd.to_datetime(frame['Journal Date'])


In [44]:
frame['Date of Purchase']=pd.to_datetime(frame['Date of Purchase'])
frame['Date of Expiry']=pd.to_datetime(frame['Date of Expiry'])


In [45]:
frame['Denominations']=frame['Denominations'].str.replace(',', '').astype(int)
frame.drop(['Denominations_m'],inplace=True,axis=1)

In [46]:
df = SmartDataframe(frame,config = {'llm':chat})
print(df.chat('What is the Name of the Political Party with max denominations?'))
print(df.chat('What is the total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019?'))
print(df.chat('What is the Name of the Political Party with max denominations?'))


The Name of the Political Party with max denominations is ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM.
3000000
ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM


In [47]:
from groq import Groq
client = Groq(api_key='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')
def run_groq_model(messages, model, temperature=0.7, top_p=1, max_tokens=16):
    chat_completion = client.chat.completions.create(
        messages=messages, temperature=temperature, top_p=top_p,
        model=model, n=1, max_tokens=max_tokens
    )
    return chat_completion.choices[0].message.content

In [48]:
frame.to_csv('test.csv',index=False)

frame=frame
question=f"""Question: "Write pandas code to analyze the attached {frame} to perform the following tasks:
1. What is the total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019?
2. What is the total bond amount purchased by CHOUDHARY GARMENTS on 12th April 2019?
3. What is the total number of bonds purchased by ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED on 12th April 2019?
4. What is the total amount received by AAM AADMI PARTY from DR. MANDEEP SHARMA in the year 2022?
5. What is the Name of the Political Party with max total amount recevied?
Your response should be a markdown table where each expert contributes step-by-step, explaining their logic and refining each other's suggestions."
"""

prompt = """1
You are expert in Pandas Python library, your goal is to answer your question by generating code for the data frame mentioned in your question.
The question is...

Question: """ + question

ret = run_groq_model([{'role': 'user', 'content': prompt}], "llama3-70b-8192", max_tokens=2048, temperature=0.0)
print(ret)

I'll provide the pandas code to answer each question and explain the logic behind each step.

**Question 1: What is the total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019?**

```python
import pandas as pd

# assume the dataframe is named df

telugu_desam_party_bonds = df[(df['Date of Encashment'] == '2019-04-12') & (df['Name of the Political Party'] == 'TELUGU DESAM PARTY')]

total_bond_amount = telugu_desam_party_bonds['Denominations'].sum()

print(f'Total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019: {total_bond_amount}')
```

**Question 2: What is the total bond amount purchased by CHOUDHARY GARMENTS on 12th April 2019?**

```python
choudhary_garments_bonds = df[(df['Date of Purchase'] == '2019-04-12') & (df['Name of the Purchaser'] == 'CHOUDHARY GARMENTS')]

total_bond_amount = choudhary_garments_bonds['Denominations'].sum()

print(f'Total bond amount purchased by CHOUDHARY GARMENTS on 12th April 2019: {total_bond_amount}')
```

**Question 3: What 

In [33]:
df=frame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20421 entries, 0 to 20420
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Sr No.                          20421 non-null  int64         
 1   Date of Encashment              20421 non-null  datetime64[ns]
 2   Name of the Political Party     20421 non-null  object        
 3   Account no. of
Political Party  20421 non-null  object        
 4   Prefix                          20421 non-null  object        
 5   Bond Number                     20421 non-null  int64         
 6   Denominations                   20421 non-null  int64         
 7   Pay Branch
Code                 20421 non-null  int64         
 8   Pay Teller                      20421 non-null  int64         
 9   Sr No._m                        20421 non-null  float64       
 10  Reference No (URN)              20421 non-null  object        
 11  Jo

In [40]:
frame

,Sr No.,Date of Encashment,Name of the Political Party,Account no. of\nPolitical Party,Prefix,Bond Number,Denominations,Pay Branch\nCode,Pay Teller,Sr No._m,Reference No (URN),Journal Date,Date of Purchase,Date of Expiry,Name of the Purchaser,Issue Branch Code,Issue Teller,Status
0,1,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,775,10000000,800,2770121,0.0,0,0,0,0,0,0.0,0.0,0
1,2,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3975,10000000,800,2770121,0.0,0,0,0,0,0,0.0,0.0,0
2,3,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3967,10000000,800,2770121,0.0,0,0,0,0,0,0.0,0.0,0
3,4,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,10418,1000000,800,2770121,0.0,0,0,0,0,0,0.0,0.0,0
4,5,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,126,1000000,800,2770121,0.0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20416,20417,2024-01-24,JANASENA PARTY,*******3929,TL,2619,1000000,847,7620160,18638.0,00952202401100000003723,2024-01-10 00:00:00,2024-01-10 00:00:00,2024-01-24 00:00:00,VALLURUPALLI PRABHU KISHORE,952.0,8487561.0,Paid
20417,20418,2024-01-24,JANASENA PARTY,*******3929,TL,2608,1000000,847,7620160,18639.0,00952202401100000003723,2024-01-10 00:00:00,2024-01-10 00:00:00,2024-01-24 00:00:00,VALLURUPALLI PRABHU KISHORE,952.0,8487561.0,Paid
20418,20419,2024-01-24,JANASENA PARTY,*******3929,TL,2633,1000000,847,7620160,18640.0,00952202401100000003723,2024-01-10 00:00:00,2024-01-10 00:00:00,2024-01-24 00:00:00,VALLURUPALLI PRABHU KISHORE,952.0,8487561.0,Paid
20419,20420,2024-01-24,JANASENA PARTY,*******3929,TL,2627,1000000,847,7620160,18641.0,00952202401100000003723,2024-01-10 00:00:00,2024-01-10 00:00:00,2024-01-24 00:00:00,VALLURUPALLI PRABHU KISHORE,952.0,8487561.0,Paid


In [49]:
df=frame
telugu_desam_party_encashment = df[(df['Date of Encashment'] == '2019-04-12') & 
                                   (df['Name of the Political Party'] == 'TELUGU DESAM PARTY')]
total_encashment_amount = telugu_desam_party_encashment['Denominations'].sum()
print(f"Total bond amount encashed by TELUGU DESAM PARTY on 12th April 2019: {total_encashment_amount}")


choudhary_garments_purchase = df[(df['Date of Purchase'] == '2019-04-12') & 
                                   (df['Name of the Purchaser'] == 'CHOUDHARY GARMENTS')]
total_purchase_amount = choudhary_garments_purchase['Denominations'].sum()
print(total_purchase_amount)


acropolis_maintenance_services_purchase = df[(df['Date of Purchase'] == '2019-04-12') & 
                                             (df['Name of the Purchaser'] == 'ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED')]
total_bonds_purchased = len(acropolis_maintenance_services_purchase)
print(f"Total number of bonds purchased by ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED on 12th April 2019: {total_bonds_purchased}")


aam_aadmi_party_receipts = df[(df['Date of Encashment'].dt.year == 2022) & 
                             (df['Name of the Political Party'] == 'AAM AADMI PARTY') & 
                             (df['Name of the Purchaser'] == 'DR. MANDEEP SHARMA')]
total_receipt_amount = aam_aadmi_party_receipts['Denominations'].sum()
print(f"Total amount received by AAM AADMI PARTY from DR. MANDEEP SHARMA in the year 2022: {total_receipt_amount}")


party_wise_amount_received = df.groupby('Name of the Political Party')['Denominations'].sum().reset_index()
# Find the party with the maximum total amount received
max_amount_received_party = party_wise_amount_received.loc[party_wise_amount_received['Denominations'].idxmax()]
print(f'Name of the Political Party with max total amount received: {max_amount_received_party["Name of the Political Party"]}')

Total bond amount encashed by TELUGU DESAM PARTY on 12th April 2019: 3000000
2000000
Total number of bonds purchased by ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED on 12th April 2019: 7
Total amount received by AAM AADMI PARTY from DR. MANDEEP SHARMA in the year 2022: 1000000
Name of the Political Party with max total amount received: BHARATIYA JANATA PARTY


In [58]:
while True:
    frame=frame
    input_by_user=str(input())
    question=f"""
    Question: "Write pandas code to answer the below question by carefully going through the columns and rows of the dataframe {frame}.The meaning of columns is mentioned below:
    
    'Sr No.': 'Serial number or sequence number of the entry.',
    'Date of Encashment': 'Date when the electoral bond was encashed.',
    'Name of the Political Party': 'Name of the political party receiving the electoral bond.',
    'Account no. of\nPolitical Party': 'Account number of the political party where the bond amount is deposited.',
    'Prefix': 'Prefix code for the type or category of bond.',
    'Bond Number': 'Unique identification number assigned to the electoral bond.',
    'Denominations': 'Face value or denomination of the electoral bond in Indian Rupees.',
    'Pay Branch\nCode': 'Branch code of the bank where the bond was encashed.',
    'Pay Teller': 'Name or identifier of the teller who processed the bond encashment.',
    'Reference No (URN)': 'Unique Reference Number assigned to the transaction.',
    'Journal Date': 'Date when the transaction was recorded in the bank\'s journal.',
    'Date of Purchase': 'Date when the electoral bond was purchased by the buyer.',
    'Date of Expiry': 'Date when the electoral bond expires or becomes invalid if not used.',
    'Name of the Purchaser': 'Name of the individual or entity purchasing the electoral bond.',
    'Issue Branch Code': 'Branch code of the bank that issued the electoral bond.',
    'Issue Teller': 'Name or identifier of the teller who issued the electoral bond.',
    'Status': 'Current status or state of the electoral bond (e.g., encashed, expired, etc.)'
    
    Also keep in mind that these are the Party names in the frame:-
    ['ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM',
       'BHARAT RASHTRA SAMITHI', 'BHARATIYA JANATA PARTY',
       'PRESIDENT, ALL INDIA CONGRESS COMMITTEE', 'SHIVSENA',
       'TELUGU DESAM PARTY',
       'YSR CONGRESS PARTY (YUVAJANA SRAMIKA RYTHU CONGRESS PARTY)',
       'DRAVIDA MUNNETRA KAZHAGAM (DMK)', 'JANATA DAL ( SECULAR )',
       'NATIONALIST CONGRESS PARTY MAHARASHTRA PRADESH',
       'ALL INDIA TRINAMOOL CONGRESS', 'BIHAR PRADESH JANTA DAL(UNITED)',
       'RASHTRIYA JANTA DAL', 'AAM AADMI PARTY',
       'ADYAKSHA SAMAJVADI PARTY', 'SHIROMANI AKALI DAL',
       'JHARKHAND MUKTI MORCHA', 'JAMMU AND KASHMIR NATIONAL CONFERENCE',
       'BIJU JANATA DAL', 'GOA FORWARD PARTY',
       'MAHARASHTRAWADI GOMNTAK PARTY', 'SIKKIM KRANTIKARI MORCHA',
       'JANASENA PARTY', 'SIKKIM DEMOCRATIC FRONT']
       
    make a match for the party name mentioned in the question and select the most suitable party name from the
    above list whereever necessary
    {input_by_user}
    
    Your response should be a code to be executed in '''  ''' and print just the final answer.
    """
    prompt = """
    You are expert in Pandas Python library, your goal is to answer your question by generating code for the data frame mentioned in your question.
    The question is...

    Question: """ + question

    ret = run_groq_model([{'role': 'user', 'content': prompt}], "llama3-70b-8192", max_tokens=2048, temperature=0.0)
    print(ret)
    text=ret
    start_pos = text.find('```')
    end_pos = text.find('```', start_pos + 3)  # Start searching from position after first triple backticks
    
    code_block = text[start_pos + 3:end_pos].strip()
    
    try:
        exec(code_block)
    except Exception as e:
        print(f"Error occurred while executing the code block:\n{e}")

 What is the total number of bonds purchased by ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED on 12th April 2019?


Here is the code to answer the question:
```
import pandas as pd

# assume the dataframe is already loaded into a variable called 'df'

# filter the dataframe to get the rows where the purchaser is ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED
# and the date of purchase is 2019-04-12
filtered_df = df[(df['Name of the Purchaser'] == 'ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED') & 
                  (df['Date of Purchase'] == '2019-04-12')]

# count the number of bonds purchased
total_bonds = filtered_df.shape[0]

print(total_bonds)
```
Note: Since the party name mentioned in the question is not present in the list of party names provided, I assume it's a purchaser name. If it's a party name, you can modify the code accordingly.
7


 What is the total amount received by AAM AADMI PARTY from DR. MANDEEP SHARMA in the year 2022?


Here is the code to answer the question:
```
import pandas as pd

# assume the dataframe is already loaded into a variable called df

party_name = 'AAM AADMI PARTY'
purchaser_name = 'DR. MANDEEP SHARMA'
year = 2022

result = df[(df['Name of the Political Party'] == party_name) &
             (df['Name of the Purchaser'] == purchaser_name) &
             (df['Date of Encashment'].dt.year == year)]

total_amount = result['Denominations'].sum()

print(total_amount)
```
This code filters the dataframe to only include rows where the political party is 'AAM AADMI PARTY', the purchaser is 'DR. MANDEEP SHARMA', and the year of encashment is 2022. It then sums up the 'Denominations' column for these rows to get the total amount received.
1000000


KeyboardInterrupt: Interrupted by user

In [ ]:
def response_generator(question):
    answer = 
    yield answer

In [68]:
def response_generator(user_input):
    question=f"""
    Question: "Write pandas code to answer the below question by carefully going through the columns and rows of the dataframe {frame}.The meaning of columns is mentioned below:
    
    'Sr No.': 'Serial number or sequence number of the entry.',
    'Date of Encashment': 'Date when the electoral bond was encashed.',
    'Name of the Political Party': 'Name of the political party receiving the electoral bond.',
    'Account no. of\nPolitical Party': 'Account number of the political party where the bond amount is deposited.',
    'Prefix': 'Prefix code for the type or category of bond.',
    'Bond Number': 'Unique identification number assigned to the electoral bond.',
    'Denominations': 'Face value or denomination of the electoral bond in Indian Rupees.',
    'Pay Branch\nCode': 'Branch code of the bank where the bond was encashed.',
    'Pay Teller': 'Name or identifier of the teller who processed the bond encashment.',
    'Reference No (URN)': 'Unique Reference Number assigned to the transaction.',
    'Journal Date': 'Date when the transaction was recorded in the bank\'s journal.',
    'Date of Purchase': 'Date when the electoral bond was purchased by the buyer.',
    'Date of Expiry': 'Date when the electoral bond expires or becomes invalid if not used.',
    'Name of the Purchaser': 'Name of the individual or entity purchasing the electoral bond.',
    'Issue Branch Code': 'Branch code of the bank that issued the electoral bond.',
    'Issue Teller': 'Name or identifier of the teller who issued the electoral bond.',
    'Status': 'Current status or state of the electoral bond (e.g., encashed, expired, etc.)'
    
    Also keep in mind that these are the Party names in the frame:-
    ['ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM',
       'BHARAT RASHTRA SAMITHI', 'BHARATIYA JANATA PARTY',
       'PRESIDENT, ALL INDIA CONGRESS COMMITTEE', 'SHIVSENA',
       'TELUGU DESAM PARTY',
       'YSR CONGRESS PARTY (YUVAJANA SRAMIKA RYTHU CONGRESS PARTY)',
       'DRAVIDA MUNNETRA KAZHAGAM (DMK)', 'JANATA DAL ( SECULAR )',
       'NATIONALIST CONGRESS PARTY MAHARASHTRA PRADESH',
       'ALL INDIA TRINAMOOL CONGRESS', 'BIHAR PRADESH JANTA DAL(UNITED)',
       'RASHTRIYA JANTA DAL', 'AAM AADMI PARTY',
       'ADYAKSHA SAMAJVADI PARTY', 'SHIROMANI AKALI DAL',
       'JHARKHAND MUKTI MORCHA', 'JAMMU AND KASHMIR NATIONAL CONFERENCE',
       'BIJU JANATA DAL', 'GOA FORWARD PARTY',
       'MAHARASHTRAWADI GOMNTAK PARTY', 'SIKKIM KRANTIKARI MORCHA',
       'JANASENA PARTY', 'SIKKIM DEMOCRATIC FRONT']
       
    make a match for the party name mentioned in the question and select the most suitable party name from the
    above list whereever necessary
    {user_input}
    
    Your response should be a code to be executed in '''  ''' and print just the final answer.
    """
    prompt = """
    You are expert in Pandas Python library, your goal is to answer your question by generating code for the data frame mentioned in your question.
    The question is...

    Question: """ + question

    ret = run_groq_model([{'role': 'user', 'content': prompt}], "llama3-70b-8192", max_tokens=2048, temperature=0.0)
    return ret

In [70]:
response_generator('What is the total amount received by AAM AADMI PARTY from DR. MANDEEP SHARMA in the year 2022?')

"Here is the code to answer the question:\n```\nimport pandas as pd\n\n# assume the dataframe is already loaded into a variable called df\n\nparty_name = 'AAM AADMI PARTY'\npurchaser_name = 'DR. MANDEEP SHARMA'\nyear = 2022\n\nresult = df[(df['Name of the Political Party'] == party_name) &\n             (df['Name of the Purchaser'] == purchaser_name) &\n             (df['Date of Encashment'].dt.year == year)]\n\ntotal_amount = result['Denominations'].sum()\n\nprint(total_amount)\n```\nThis code filters the dataframe to only include rows where the political party is 'AAM AADMI PARTY', the purchaser is 'DR. MANDEEP SHARMA', and the year of encashment is 2022. It then sums up the 'Denominations' column for these rows to get the total amount received."

In [66]:
aam_aadmi_party_df = df[df['Name of the Political Party'] == 'AAM AADMI PARTY']

# filter the dataframe to only include rows where the purchaser name is 'DR. MANDEEP SHARMA'
mandeep_sharma_df = aam_aadmi_party_df[aam_aadmi_party_df['Name of the Purchaser'] == 'DR. MANDEEP SHARMA']

# filter the dataframe to only include rows where the year of encashment is 2022
year_2022_df = mandeep_sharma_df[mandeep_sharma_df['Date of Encashment'].dt.year == 2022]

# calculate the total amount received
total_amount = year_2022_df['Denominations'].sum()

print(total_amount)

1000000


In [69]:
filtered_df = df[(df['Date of Purchase'] == '2019-04-12') & (df['Name of the Purchaser'] == 'CHOUDHARY GARMENTS')]

# sum the 'Denominations' column to get the total bond amount purchased
total_bond_amount = filtered_df['Denominations'].sum()

print(total_bond_amount)

2000000


In [119]:
def code(frame,text):
    df=frame
    start_pos = text.find('```')
    end_pos = text.find('```', start_pos + 3)  # Start searching from position after first triple backticks
    answer=[]
    code_block = text[start_pos + 3:end_pos].strip()
#     print(code_block)
#     print(type(code_block))
#     print(type(exec(code_block)))
    answer.append())
    return answer

In [120]:
code(df,response_generator('What is the total amount received by AAM AADMI PARTY from DR. MANDEEP SHARMA in the year 2022?'))

1000000


[None]

In [146]:
def code(frame, text):
    df = frame
    start_pos = text.find('```')
    end_pos = text.find('```', start_pos + 3)  # Start searching from position after first triple backticks
    answer = []
    
    if start_pos == -1 or end_pos == -1:
        raise ValueError("Code block not found in the text")
    
    code_block = text[start_pos + 3:end_pos].strip()

    # Create a local context dictionary
    local_vars = {'df': df}

    # Execute the code block within the provided local context
    exec(code_block, {}, local_vars)

    # Add any variables from the local context to the answer list
    for var in local_vars:
        if var != 'df':  # Exclude the input variable 'df'
            answer.append(local_vars[var])
    
    return answer[-1]

frame=df
text=response_generator('What is the Name of the Political Party with max total amount recevied?')

out=code(df,text)

BHARATIYA JANATA PARTY


[<module 'pandas' from '/opt/conda/lib/python3.10/site-packages/pandas/__init__.py'>,
                           Name of the Political Party  Denominations
 0                                     AAM AADMI PARTY      654500000
 1                            ADYAKSHA SAMAJVADI PARTY      140500000
 2            ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM       60500000
 3                        ALL INDIA TRINAMOOL CONGRESS    16095314000
 4                              BHARAT RASHTRA SAMITHI    12147099000
 5                              BHARATIYA JANATA PARTY    60605111000
 6                     BIHAR PRADESH JANTA DAL(UNITED)      140000000
 7                                     BIJU JANATA DAL     7755000000
 8                     DRAVIDA MUNNETRA KAZHAGAM (DMK)     6390000000
 9                                   GOA FORWARD PARTY        3500000
 10              JAMMU AND KASHMIR NATIONAL CONFERENCE        5000000
 11                                     JANASENA PARTY      210000000
 12 

In [147]:
def main(input_text_path):
    with open(input_text_path, 'r') as file:
        questions = file.readlines()
    
    processed_answers = [
        str(code(df, response_generator(question)))
        for question in questions
    ]
    print(processed_answers)
    
    output_text = './answer.txt'
    with open(output_text, 'w') as file:
        for answer in processed_answers:
            print(type(answer))
            print(answer)
            file.write(str(answer) + '\n')
    
    return output_text

main('/kaggle/input/questions/questions.txt')

3000000
2000000
7
1000000
['3000000', '2000000', '7', '1000000']
<class 'str'>
3000000
<class 'str'>
2000000
<class 'str'>
7
<class 'str'>
1000000


'./answer.txt'

7


In [136]:
out

7

In [83]:
response_generator('What is the total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019?')

"Here is the code to answer the question:\n```\nimport pandas as pd\n\n# assume the dataframe is already loaded into a variable called df\n\nparty_name = 'TELUGU DESAM PARTY'\ndate_of_encashment = '2019-04-12'\n\ntotal_bond_amount = df[(df['Name of the Political Party'] == party_name) & (df['Date of Encashment'] == date_of_encashment)]['Denominations'].sum()\n\nprint(total_bond_amount)\n```\nThis code filters the dataframe to only include rows where the `Name of the Political Party` column matches `TELUGU DESAM PARTY` and the `Date of Encashment` column matches `2019-04-12`. It then sums up the `Denominations` column for these filtered rows to get the total bond amount encashed by TELUGU DESAM PARTY on 12th April 2019."